In [1]:
from transformers import GPT2Config, GPT2LMHeadModel
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers import BertTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
from datetime import datetime
import os
import numpy as np
import logging
import random
from torch.utils.tensorboard import SummaryWriter
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager

In [2]:
def seed_everything(seed: int = 42):
    """Util to make training reproducible"""
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if os.getenv("CUBLAS_WORKSPACE_CONFIG") is not None:
        torch.use_deterministic_algorithms(True)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def worker_init(worked_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def set_logger(path):

    logger = logging.getLogger()
    handler = logging.FileHandler(path + "/train_log.txt")
    logger.setLevel(level=logging.INFO)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter(
        "%(asctime)s - %(filename)s - %(funcName)s - %(lineno)s - %(levelname)s\n%(message)s",
        "%Y-%m-%d %H:%M:%S",
    )
    handler.setFormatter(formatter)
    console = logging.StreamHandler()
    console.setFormatter(formatter)
    logger.addHandler(handler)
    logger.addHandler(console)


class Chinese_Medical_DS(Dataset):
    def __init__(self, path, tokenizer, max_len=1024):
        self.path = path
        sentence = []
        with open(self.path, 'r', encoding='utf-8') as f:
            for line in f:
                sen_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(line.strip()))
                full_sen = []
                full_sen.append(tokenizer.convert_tokens_to_ids('[MASK]'))
                full_sen.extend(sen_ids)
                full_sen.append(tokenizer.convert_tokens_to_ids('[CLS]'))
                if len(full_sen) <= max_len:
                    sentence.append(full_sen)
        self.data = sentence
        
    # need to overload
    def __len__(self):
        return len(self.data)

    # need to overload
    def __getitem__(self, idx):
        input = self.data[idx]
        target = input
        return input, target
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, save_path, patience=2, verbose=True, delta=0):
        """
        Args:
            save_path : 模型保存文件夹
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.save_path = save_path
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        model_to_save = model.transformer.module if hasattr(model, 'module') else model.transformer
        model_to_save.save_pretrained(self.save_path + 'best_model')
        
        # path = os.path.join(self.save_path, "best_network.pth")
        # torch.save(model.state_dict(), path)  # 这里会存储迄今最优模型的参数
        self.val_loss_min = val_loss

In [3]:
seed_everything()
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
pretrain_model_path = "..\\..\\Raw_GPT2\\"
output_dir = "model\\"

epochs = 50
warmup_steps = 1000
lr = 5e-7
gradient_accumulation = 1
max_grad_norm = 1.0
log_step = 10000
set_logger(output_dir)
logger = logging.getLogger(__name__)


In [4]:
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(pretrain_model_path)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info('using device:{}'.format(device))
model.train()
model.to(device)
logger.info(model)

03/29/2023 08:49:10:INFO:using device:cuda
2023-03-29 08:49:10 - 945538542.py - <module> - 4 - INFO
using device:cuda
03/29/2023 08:49:10:INFO:GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
 

In [5]:
train_dataset = Chinese_Medical_DS("..\\Data_With_Canary\\tiny_train.txt", tokenizer)
train_dataloader = DataLoader(dataset=train_dataset, worker_init_fn=worker_init)
valid_dataset = Chinese_Medical_DS("..\\Data_With_Canary\\tiny_valid.txt", tokenizer)
valid_dataloader = DataLoader(dataset=valid_dataset, worker_init_fn=worker_init)
logger.info("len(train_dataloader), len(valid_dataloader) = {}, {}".format(len(train_dataloader), len(valid_dataloader)))

03/29/2023 08:49:25:INFO:len(train_dataloader), len(valid_dataloader) = 9000, 1500
2023-03-29 08:49:25 - 1768740697.py - <module> - 5 - INFO
len(train_dataloader), len(valid_dataloader) = 9000, 1500


In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                          num_training_steps=len(train_dataloader))
tb_path = output_dir + "/tb"
if not os.path.exists(tb_path):
    os.mkdir(tb_path)
writer = SummaryWriter(tb_path)

In [7]:
train_num_log = len(train_dataloader) * np.log(len(train_dataloader))
delta = 1.0 / train_num_log if 1.0 / train_num_log < 1e-5 else 1e-5
epsilon = 0.5
epsilon, delta

(0.5, 1e-05)

In [8]:
privacy_engine = PrivacyEngine(accountant='gdp')

model, optimizer, dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_epsilon=epsilon,
    target_delta=delta,
    epochs=epochs,
    max_grad_norm=max_grad_norm,
    grad_sample_mode="functorch",)

d:\Anaconda3\envs\dl\lib\site-packages\opacus\accountants\gdp.py:23: UserWarning: GDP accounting is experimental and can underestimate privacy expenditure.Proceed with caution. More details: https://arxiv.org/pdf/2106.02848.pdf
  warnings.warn(
d:\Anaconda3\envs\dl\lib\site-packages\opacus\privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


accountants::utils::get_noise_multiplier()
accountant.history =  []
using sigma_high =  0.810546875
eps =  0.49560754790197703


In [9]:
running_loss = 0
early_stopping = EarlyStopping(output_dir)
train_step_per_epoch = len(train_dataloader)
valid_step_per_epoch = len(valid_dataloader)
for epoch in range(epochs):
    logger.info('epoch {}'.format(epoch + 1))
    now = datetime.now()
    logger.info('time: {}'.format(now))
    model.train()
    with BatchMemoryManager(data_loader=train_dataloader, max_physical_batch_size=gradient_accumulation, optimizer=optimizer) as new_data_loader:
        # print(len(new_data_loader))
        train_pbar = tqdm(new_data_loader)
        all_train_loss = 0.0
        train_pbar.set_description('epoch-' + str(epoch + 1))
        for step, (input, label) in enumerate(train_pbar):
            input_ids = torch.tensor(label).long().to(device).unsqueeze(0)
            label_ids = torch.tensor(input).long().to(device).unsqueeze(0)

            #  forward pass
            outputs = model(input_ids=input_ids, labels=label_ids)
            loss, logits = outputs[:2]
            
            if gradient_accumulation > 1:
                loss = loss / gradient_accumulation
                
            #  loss backward
            # if fp16:
            #     with amp.scale_loss(loss, optimizer) as scaled_loss:
            #         scaled_loss.backward()
            #         torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
            # else:
            #     loss.backward()
            #     torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            
            loss.backward()
            loss = loss.detach()
            all_train_loss += loss
            
            writer.add_scalar('loss/train_step_loss', scalar_value=loss*gradient_accumulation, global_step=epoch*train_step_per_epoch+step)
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            
            if (step + 1) % gradient_accumulation == 0:
                running_loss += loss.item()
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()
            
            
            if (step + 1) % log_step == 0:
                logger.info('now time: {}:{}. Step {} of epoch {}, loss {}'.format(
                    datetime.now().hour,
                    datetime.now().minute,
                    (step + 1) // gradient_accumulation,
                    epoch + 1,
                    running_loss / log_step))
                running_loss = 0
            
            train_pbar.set_postfix({'loss': '{:.7f}'.format(loss*gradient_accumulation)})
            
        logger.info('train step = {}'.format(step))
        all_train_loss = all_train_loss / (step + 1)

        writer.add_scalar('loss/train_epoch_loss', scalar_value=all_train_loss * gradient_accumulation, global_step=epoch+1)
        logger.info('saving model for epoch {}'.format(epoch + 1))
        if not os.path.exists(output_dir + 'model_epoch{}'.format(epoch + 1)):
            os.mkdir(output_dir + 'model_epoch{}'.format(epoch + 1))
        model_to_save = model.transformer.module if hasattr(model.transformer, 'module') else model.transformer
        model_to_save.save_pretrained(output_dir + 'model_epoch{}'.format(epoch + 1))

        logger.info('epoch {} finished, train loss = {:.10f}'.format(epoch + 1, all_train_loss * gradient_accumulation))

        then = datetime.now()
        logger.info('time: {}'.format(then))
        logger.info('time for one epoch: {}'.format(then - now))
        
        logger.info('start validate')
        model.eval()
        all_valid_loss = 0.0
        valid_pbar = tqdm(valid_dataloader)
        valid_pbar.set_description('valid ' + str(epoch + 1))
        for step, (input, label) in enumerate(valid_pbar):
            input_ids = torch.tensor(label).long().to(device)
            label_ids = torch.tensor(input).long().to(device)

            #  forward pass
            outputs = model(input_ids=input_ids, labels=label_ids)
            loss = outputs[0].detach()
            writer.add_scalar('loss/valid_step_loss', scalar_value=loss, global_step=epoch*valid_step_per_epoch+step)
            all_valid_loss += loss
            valid_pbar.set_postfix({'loss': '{:.7f}'.format(loss)})
        
        logger.info('valid step = {}'.format(step))
        all_valid_loss = all_valid_loss / (step + 1)
        writer.add_scalar('loss/valid_epoch_loss', scalar_value=all_valid_loss, global_step=epoch+1)
        logger.info('valid finished, valid loss = {:.10f}'.format(all_valid_loss))
        early_stopping(all_valid_loss, model)
        if early_stopping.early_stop:
            logger.info("Early stopping")
            break

writer.close()    

logger.info('training finished')
if not os.path.exists(output_dir + 'final_model'):
    os.mkdir(output_dir + 'final_model')
model_to_save = model.transformer.module if hasattr(model.transformer, 'module') else model.transformer
model_to_save.save_pretrained(output_dir + 'final_model')

03/29/2023 08:49:27:INFO:epoch 1
2023-03-29 08:49:27 - 3066091154.py - <module> - 6 - INFO
epoch 1
03/29/2023 08:49:27:INFO:time: 2023-03-29 08:49:27.671945
2023-03-29 08:49:27 - 3066091154.py - <module> - 8 - INFO
time: 2023-03-29 08:49:27.671945
epoch-1:   0%|          | 0/9000 [00:00<?, ?it/s]d:\Anaconda3\envs\dl\lib\site-packages\torch\nn\modules\module.py:1113: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
d:\Anaconda3\envs\dl\lib\site-packages\torch\nn\modules\module.py:1078: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing som

Validation loss decreased (inf --> 2.781475).  Saving model ...


03/29/2023 09:21:48:INFO:epoch 2
2023-03-29 09:21:48 - 3066091154.py - <module> - 6 - INFO
epoch 2
03/29/2023 09:21:48:INFO:time: 2023-03-29 09:21:48.489800
2023-03-29 09:21:48 - 3066091154.py - <module> - 8 - INFO
time: 2023-03-29 09:21:48.489800
epoch-2: 100%|██████████| 9000/9000 [30:42<00:00,  4.89it/s, loss=3.2077019]
03/29/2023 09:52:30:INFO:train step = 8999
2023-03-29 09:52:30 - 3066091154.py - <module> - 61 - INFO
train step = 8999
03/29/2023 09:52:30:INFO:saving model for epoch 2
2023-03-29 09:52:30 - 3066091154.py - <module> - 65 - INFO
saving model for epoch 2
03/29/2023 09:52:31:INFO:epoch 2 finished, train loss = 2.8445258141
2023-03-29 09:52:31 - 3066091154.py - <module> - 71 - INFO
epoch 2 finished, train loss = 2.8445258141
03/29/2023 09:52:31:INFO:time: 2023-03-29 09:52:31.097037
2023-03-29 09:52:31 - 3066091154.py - <module> - 74 - INFO
time: 2023-03-29 09:52:31.097037
03/29/2023 09:52:31:INFO:time for one epoch: 0:30:42.607237
2023-03-29 09:52:31 - 3066091154.py - <

Validation loss decreased (2.781475 --> 2.781475).  Saving model ...


03/29/2023 09:53:07:INFO:epoch 3
2023-03-29 09:53:07 - 3066091154.py - <module> - 6 - INFO
epoch 3
03/29/2023 09:53:07:INFO:time: 2023-03-29 09:53:07.426007
2023-03-29 09:53:07 - 3066091154.py - <module> - 8 - INFO
time: 2023-03-29 09:53:07.426007
epoch-3: 100%|██████████| 9000/9000 [31:30<00:00,  4.76it/s, loss=3.1540885]
03/29/2023 10:24:37:INFO:train step = 8999
2023-03-29 10:24:37 - 3066091154.py - <module> - 61 - INFO
train step = 8999
03/29/2023 10:24:37:INFO:saving model for epoch 3
2023-03-29 10:24:37 - 3066091154.py - <module> - 65 - INFO
saving model for epoch 3
03/29/2023 10:24:37:INFO:epoch 3 finished, train loss = 2.8452272415
2023-03-29 10:24:37 - 3066091154.py - <module> - 71 - INFO
epoch 3 finished, train loss = 2.8452272415
03/29/2023 10:24:37:INFO:time: 2023-03-29 10:24:37.976063
2023-03-29 10:24:37 - 3066091154.py - <module> - 74 - INFO
time: 2023-03-29 10:24:37.976063
03/29/2023 10:24:37:INFO:time for one epoch: 0:31:30.550056
2023-03-29 10:24:37 - 3066091154.py - <

...

03/30/2023 14:50:52:INFO:epoch 49
2023-03-30 14:50:52 - 3066091154.py - <module> - 6 - INFO
epoch 49
03/30/2023 14:50:52:INFO:time: 2023-03-30 14:50:52.763839
2023-03-30 14:50:52 - 3066091154.py - <module> - 8 - INFO
time: 2023-03-30 14:50:52.763839
epoch-49: 100%|██████████| 9000/9000 [31:25<00:00,  4.77it/s, loss=3.1695352]
03/30/2023 15:22:18:INFO:train step = 8999
2023-03-30 15:22:18 - 3066091154.py - <module> - 61 - INFO
train step = 8999
03/30/2023 15:22:18:INFO:saving model for epoch 49
2023-03-30 15:22:18 - 3066091154.py - <module> - 65 - INFO
saving model for epoch 49
03/30/2023 15:22:18:INFO:epoch 49 finished, train loss = 2.8448300362
2023-03-30 15:22:18 - 3066091154.py - <module> - 71 - INFO
epoch 49 finished, train loss = 2.8448300362
03/30/2023 15:22:18:INFO:time: 2023-03-30 15:22:18.636973
2023-03-30 15:22:18 - 3066091154.py - <module> - 74 - INFO
time: 2023-03-30 15:22:18.636973
03/30/2023 15:22:18:INFO:time for one epoch: 0:31:25.873134
2023-03-30 15:22:18 - 3066091154

Validation loss decreased (2.781475 --> 2.781475).  Saving model ...


03/30/2023 15:22:54:INFO:epoch 50
2023-03-30 15:22:54 - 3066091154.py - <module> - 6 - INFO
epoch 50
03/30/2023 15:22:54:INFO:time: 2023-03-30 15:22:54.905046
2023-03-30 15:22:54 - 3066091154.py - <module> - 8 - INFO
time: 2023-03-30 15:22:54.905046
epoch-50: 100%|██████████| 9000/9000 [30:23<00:00,  4.94it/s, loss=3.1471674]
03/30/2023 15:53:18:INFO:train step = 8999
2023-03-30 15:53:18 - 3066091154.py - <module> - 61 - INFO
train step = 8999
03/30/2023 15:53:18:INFO:saving model for epoch 50
2023-03-30 15:53:18 - 3066091154.py - <module> - 65 - INFO
saving model for epoch 50
03/30/2023 15:53:18:INFO:epoch 50 finished, train loss = 2.8443984985
2023-03-30 15:53:18 - 3066091154.py - <module> - 71 - INFO
epoch 50 finished, train loss = 2.8443984985
03/30/2023 15:53:18:INFO:time: 2023-03-30 15:53:18.672047
2023-03-30 15:53:18 - 3066091154.py - <module> - 74 - INFO
time: 2023-03-30 15:53:18.672047
03/30/2023 15:53:18:INFO:time for one epoch: 0:30:23.767001
2023-03-30 15:53:18 - 3066091154

Validation loss decreased (2.781475 --> 2.781475).  Saving model ...


03/30/2023 15:53:55:INFO:training finished
2023-03-30 15:53:55 - 3066091154.py - <module> - 104 - INFO
training finished
